In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pymongo
import time
import re
import pandas as pd

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
db = client.nasa_mars_db
collection = db.mars_news

In [4]:
browser = Browser('chrome')

In [5]:
nasa_url = "https://mars.nasa.gov/news/"
browser.visit(nasa_url)
time.sleep(0.5)

In [6]:
html_1 = browser.html
soup = BeautifulSoup(html_1, 'html.parser')

news_title =  soup.find('div', class_= 'content_title').text
# print (news_title.text)
news_para =  soup.find('div', class_= 'article_teaser_body').text
# print (news_para.text)

In [7]:
jpl_nasa_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_nasa_url)
time.sleep(0.5)

In [8]:
browser.click_link_by_id('full_image')
time.sleep(2)

In [9]:
browser.click_link_by_partial_text('more info')
time.sleep(0.5)

In [10]:
html_2 = browser.html
soup = BeautifulSoup(html_2, 'html.parser')

In [11]:
images = soup.find_all('div', class_="download_tiff")
for img in images:
    try:
        image_1 = img.a
        image_2 = image_1['href']
#         print(image_2)
        if image_2.endswith('jpg'):
            featured_image_url = image_2
#             print(featured_image_url)
            break
    except AttributeError as e:
        print (e)

In [12]:
mars_weather_twitter_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(mars_weather_twitter_url)
time.sleep(0.5)
html_3 = browser.html
soup = BeautifulSoup(html_3, 'html.parser')

In [13]:
if soup.find(href=re.compile("MarsWxReport")):
    mars_twitter_post = soup.find('p', class_= 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')
    mars_weather_twitter_post = mars_twitter_post.contents[0]
#     print(mars_weather_twitter_post)    

In [14]:
mars_facts_url = "https://space-facts.com/mars/"
browser.visit(mars_facts_url)
time.sleep(0.5)
html_4 = browser.html
soup = BeautifulSoup(html_4, 'html.parser')

In [15]:
facts_table = soup.find('table')

In [16]:
column1 = facts_table.find_all('td', class_='column-1')
column2 = facts_table.find_all('td', class_='column-2')

Parameters = []
Values = []

for row in column1:
    parameter = row.text.strip(":")
    Parameters.append(parameter)
    
for row in column2:
    value = row.text.strip()
    Values.append(value)
    
mars_facts = pd.DataFrame({
    "Parameter":Parameters,
    "Value":Values
    })

mars_facts_html = mars_facts.to_html(header=False, index=None)
# print (mars_facts)

In [17]:
mars_hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(mars_hemispheres_url)
time.sleep(0.5)
html_5 = browser.html
soup = BeautifulSoup(html_5, 'html.parser')

In [18]:
hemisphere_image_urls = []
base_url = "https://astrogeology.usgs.gov"
links = soup.find_all('div', class_="description")
for link in links:
    time.sleep(0.5)
    title = link.h3.text
#     print (title)
    new_link = base_url + str(link.a['href'])
#     print(new_link)
    browser.visit(new_link)
    time.sleep(0.5)
    browser.click_link_by_partial_href('#open')
    time.sleep(0.5)
    html_6 = browser.html
    soup = BeautifulSoup(html_6, 'html.parser')
    full_img = soup.find('img', class_ = 'wide-image')
    img_link = full_img['src']
#     print(img_link)
    full_img_link = base_url + img_link
#     print(full_img_link)
    hemisphere_image_urls.append( {'title':title, 'img_url':full_img_link} )
    browser.back()

# hemisphere_image_urls
browser.quit()

In [19]:
mars_scraped_data = {
    'Mars_Headlines' : news_title,
    'Mars_News_Details' : news_para,
    'Mars_Space_Image' : featured_image_url,
    'Mars_Weather_Report' : mars_weather_twitter_post,
    'Mars_Facts' : mars_facts_html,
    'Mars_Hemispheres' : hemisphere_image_urls
}
print (mars_scraped_data)

{'Mars_Headlines': "NASA's InSight Detects First Likely 'Quake' on Mars", 'Mars_News_Details': 'While their causes are still unknown, one of three shaking events looks a lot like the quakes detected on the Moon by the Apollo missions.', 'Mars_Space_Image': '//photojournal.jpl.nasa.gov/jpeg/PIA17838.jpg', 'Mars_Weather_Report': 'InSight sol 143 (2019-04-22) low -98.0ºC (-144.3ºF) high -19.8ºC (-3.7ºF)\nwinds from the SW at 4.5 m/s (10.1 mph) gusting to 12.1 m/s (27.2 mph)\npressure at 7.40 hPa', 'Mars_Facts': '<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <td>Moons</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n   